# Tutorial bookkeeping

In this notebook, we explain how to use the LaCE bookkeeping

In [1]:
from lace.archive.gadget_archive import GadgetArchive
from lace.archive.nyx_archive import NyxArchive

### Read full post-processing for each archive

We load all archives here, to avoid redoing these multiple times in the notebook. Then we test them one by one.

In [2]:
%%time
pedersen21_archive = GadgetArchive(postproc='Pedersen21')

CPU times: user 4.01 s, sys: 299 ms, total: 4.3 s
Wall time: 8.81 s


In [3]:
%%time
cabayol23_archive = GadgetArchive(postproc='Cabayol23')

CPU times: user 36.8 s, sys: 2.44 s, total: 39.3 s
Wall time: 1min 11s


In [4]:
%%time
# specify here the path to the Nyx files in your local computer (or set NYX_PATH environ variable)
nyx_fname = None # use NYX_PATH
nyx_archive = NyxArchive(file_name=nyx_fname,verbose=True)

read Nyx sim bar_ic_grid_3
read Nyx sim cosmo_grid_0
read Nyx sim cosmo_grid_1
read Nyx sim cosmo_grid_10
read Nyx sim cosmo_grid_11
read Nyx sim cosmo_grid_12
read Nyx sim cosmo_grid_13
read Nyx sim cosmo_grid_2
read Nyx sim cosmo_grid_3
read Nyx sim cosmo_grid_4
read Nyx sim cosmo_grid_5
read Nyx sim cosmo_grid_6
read Nyx sim cosmo_grid_7
read Nyx sim cosmo_grid_8
read Nyx sim cosmo_grid_9
read Nyx sim fiducial
read Nyx sim wdm_3.5kev_grid_1
CPU times: user 42.8 s, sys: 5.88 s, total: 48.7 s
Wall time: 1min


### Access training data

#### Pedersen21

In [5]:
archive = pedersen21_archive
# (hypercube + special pairs) * nphases * naxes * nscalings * nsnaps
# some special pairs have only 1 scaling
nelem = (30 + 1) * 2 * 1 * 3 * 11 + 6 * 2 * 1 * 1 * 11
print(len(archive.data), nelem)

emu_params = [
    "Delta2_p",
    "n_p",
    "mF",
    "sigT_Mpc",
    "gamma",
    "kF_Mpc",
]
archive_training = archive.get_training_data(emu_params)
# hypercube * nsnaps
nelem = 30 * 11
print(len(archive_training), nelem)

2178 2178
330 330


#### Cabayol23

In [6]:
archive = cabayol23_archive
# (hypercube + special pairs) * nphases * naxes * nscalings * nsnaps
nelem = (30 + 7) * 2 * 3 * 5 * 11
print(len(archive.data), nelem)

emu_params = [
    "Delta2_p",
    "n_p",
    "mF",
    "sigT_Mpc",
    "gamma",
    "kF_Mpc",
]
archive_training = archive.get_training_data(emu_params)
# hypercube * nphases * naxes * nscalings * nsnaps
n_av_phases = 30 * 1 * 3 * 5 * 11
n_av_axes   = 30 * 2 * 1 * 5 * 11
n_av_all    = 30 * 1 * 1 * 5 * 11
print(len(archive_training), n_av_phases + n_av_axes + n_av_all)

12210 12210
9900 9900


#### Nyx

In [7]:
archive = nyx_archive
print(len(archive.data))

# parameters that must be available for each training element
emu_params = [
    "Delta2_p",
    "n_p",
    "mF",
    "sigT_Mpc",
    "gamma",
    "lambda_P",
]

archive_training = archive.get_training_data(emu_params)
print(len(archive_training))

14490
4287


### Leave-one-out dropping of sims (for first 3 sims)

#### Pedersen21

In [8]:
archive = pedersen21_archive
emu_params = [
    "Delta2_p",
    "n_p",
    "mF",
    "sigT_Mpc",
    "gamma",
    "kF_Mpc",
]
for ii in range(3):
    archive_training = archive.get_training_data(emu_params, drop_sim='mpg_'+str(ii))
    # hypercube * nphases * naxes * nscalings * nsnaps
    nelem = (30-1) * 1 * 1 * 1 * 11
    print(len(archive_training), nelem)
    
    archive_testing = archive.get_testing_data('mpg_'+str(ii))
    nelem = 1 * 1 * 1 * 1 * 11
    print(len(archive_testing), nelem)    

319 319
11 11
319 319
11 11
319 319
11 11


#### Cabayol23

In [9]:
archive = cabayol23_archive
emu_params = [
    "Delta2_p",
    "n_p",
    "mF",
    "sigT_Mpc",
    "gamma",
    "kF_Mpc",
]
for ii in range(3):
    archive_training = archive.get_training_data(emu_params, drop_sim='mpg_'+str(ii))
    # hypercube * nphases * naxes * nscalings * nsnaps
    n_av_phases = (30-1) * 1 * 3 * 5 * 11
    n_av_axes   = (30-1) * 2 * 1 * 5 * 11
    n_av_all    = (30-1) * 1 * 1 * 5 * 11
    print(len(archive_training), n_av_phases + n_av_axes + n_av_all)
    
    archive_testing = archive.get_testing_data('mpg_'+str(ii))
    nelem = 1 * 1 * 1 * 1 * 11
    print(len(archive_testing), nelem)

9570 9570
11 11
9570 9570
11 11
9570 9570
11 11


#### Nyx

In [10]:
archive = nyx_archive
# parameters that must be available for each training element
emu_params = [
    "Delta2_p",
    "n_p",
    "mF",
    "sigT_Mpc",
    "gamma",
    "lambda_P",
]
for ii in range(3):
    archive_training = archive.get_training_data(
        emu_params=emu_params, 
        drop_sim='nyx_'+str(ii)
    )
    print(len(archive_training))
    
    archive_testing = archive.get_testing_data(
        'nyx_'+str(ii), 
        emu_params=emu_params
    )
    print(len(archive_testing))    

3980
15
3980
15
3980
15


### Read special simulations for testing

#### Cabayol23

In [11]:
archive = cabayol23_archive
for sim in archive.list_sim_test:
    archive_testing = archive.get_testing_data(sim)
    nelem = 1 * 1 * 1 * 1 * 11
    print(len(archive_testing), nelem, archive_testing[0]['sim_label'])

11 11 mpg_central
11 11 mpg_seed
11 11 mpg_growth
11 11 mpg_neutrinos
11 11 mpg_curved
11 11 mpg_running
11 11 mpg_reio


#### Pedersen21

In [12]:
archive = pedersen21_archive
for sim in archive.list_sim_test:
    archive_testing = archive.get_testing_data(sim)
    nelem = 1 * 1 * 1 * 1 * 11
    print(len(archive_testing), nelem, archive_testing[0]['sim_label'])

11 11 mpg_central
11 11 mpg_seed
11 11 mpg_growth
11 11 mpg_neutrinos
11 11 mpg_curved
11 11 mpg_running
11 11 mpg_reio


#### Nyx

In [13]:
archive = nyx_archive
for sim in archive.list_sim_test:
    # no "unscaled" measurement for this simulation
    
    if sim == 'nyx_central':
        ind_rescaling = 1
    else:
        ind_rescaling = None
    archive_testing = archive.get_testing_data(sim, ind_rescaling=ind_rescaling)
    if archive_testing:
        print(archive_testing[0]['sim_label'], len(archive_testing))
    else:
        print('empty sim:',sim)

nyx_central 15
nyx_seed 15
nyx_wdm 15
